In [ ]:
!pip install sentence-transformers
!pip install scikit-learn

In [27]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer
from google.colab import drive
import matplotlib.pyplot as plt

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [28]:
df = pd.read_csv('/content/drive/My Drive/Диплом/dataSets/rel_dataset_v1.csv')
df_2 = pd.read_csv('/content/drive/My Drive/Диплом/dataSets/rel_dataset_v2.csv')
df_2

,id,queries,tags,TFS,QLS,SSS,Relevance
0,1,How can I get chewy chocolate chip cookies?,baking cookies texture,1778,1.0,0.613218,0.466042
1,2,How should I cook bacon in an oven?,oven cooking-time bacon,548,1.0,0.490151,0.143516
2,3,What is the difference between white and brown...,eggs,430,1.0,0.207717,0.112521
3,4,What is the difference between baking soda and...,baking-soda baking-powder,72,1.0,0.352997,0.018683
4,5,"In a tomato sauce recipe, how can I cut the ac...",sauce pasta tomatoes italian-cuisine,909,0.7,0.308210,0.238074
...,...,...,...,...,...,...,...
15399,73670,Poached eggs at altitude?,eggs poaching high-altitude,460,0.5,0.239465,0.120295
15400,73678,How to thicken buttercream without adding more...,frosting,28,1.0,0.293548,0.007135
15401,73680,Looking for old Italian recipe for chamellas,baking,1444,1.0,0.330974,0.378413
15402,73681,Can i make ice cream with artificial sweetener,ice-cream,118,1.0,0.194948,0.030713


In [ ]:
# 1. Загружаем Sentence-BERT
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# 2. Преобразуем запросы и теги в эмбеддинги
query_embeddings = model.encode(df['queries'].tolist())
tag_embeddings = model.encode(df['tags'].tolist())

In [32]:
# 3. Функция для поиска одного лучшего тега
def recommend_best_tag_by_query(query, top_n=10):
    # Преобразуем запрос в эмбеддинг
    query_embedding = model.encode([query])

    # Вычисляем косинусное сходство между запросом и всеми тегами
    cos_similarities = cosine_similarity(query_embedding, tag_embeddings)

    # Находим индекс тега с максимальным значением сходства
    best_tag_idx = cos_similarities[0].argmax()

    # Получаем лучший тег, его релевантность и эмбеддинг
    best_tag = df['tags'][best_tag_idx]
    best_relevance = df['relevance'][best_tag_idx]
    best_tag_embedding = tag_embeddings[best_tag_idx]

    # Ищем похожие теги
    similar_tags = find_similar_tags(best_tag, best_tag_embedding, best_relevance, top_n)

    # Выводим лучший тег и топ-10 похожих тегов
    print(f"Запрос: '{query}'")
    print(f"Предсказанный тег для запроса: {best_tag} (Relevance: {best_relevance})")
    print("\nТоп-10 похожих тегов:")
    for tag, similarity, relevance in similar_tags:
        print(f"Тег: {tag} (Similarity: {similarity:.2f}, Relevance: {relevance})")

# 4. Функция для поиска похожих тегов
# Функция для поиска похожих тегов
def find_similar_tags(best_tag, best_tag_embedding, best_relevance, top_n=10):
    # Вычисляем косинусное сходство между лучшим тегом и всеми остальными
    tag_similarities = cosine_similarity([best_tag_embedding], tag_embeddings)[0]

    # Словарь для хранения уникальных тегов с наилучшими релевантностями
    unique_tags = {}

    # Составляем список из тегов, схожести и релевантности
    for idx, (tag, relevance) in enumerate(zip(df['tags'], df['relevance'])):
        # Пропускаем сам лучший тег и теги с релевантностью меньше или равной
        if tag == best_tag or relevance <= best_relevance:
            continue

        # Если тег еще не встречался, добавляем его
        if tag not in unique_tags:
            unique_tags[tag] = (tag_similarities[idx], relevance)
        else:
            # Если тег уже встречался, сравниваем по релевантности
            existing_similarity, existing_relevance = unique_tags[tag]
            # Если новая релевантность выше, обновляем тег
            if existing_relevance < relevance:
                unique_tags[tag] = (tag_similarities[idx], relevance)

    # Преобразуем словарь обратно в список и сортируем по схожести и релевантности
    similar_tags = [(tag, similarity, relevance) for tag, (similarity, relevance) in unique_tags.items()]

    # Сортируем по схожести, затем по релевантности
    similar_tags.sort(key=lambda x: (-x[1], -x[2]))  # Сначала по схожести, затем по релевантности

    # Возвращаем топ-N тегов
    return similar_tags[:top_n]


# Пример использования
new_query = "how do I bake soft cookies?"
recommend_best_tag_by_query(new_query, top_n=10)

Запрос: 'how do I bake soft cookies?'
Предсказанный тег для запроса: baking cookies (Relevance: 0.2182915450004952)

Топ-10 похожих тегов:
Тег: baking cookies flour (Similarity: 0.91, Relevance: 0.2722715729386034)
Тег: baking cookies texture (Similarity: 0.91, Relevance: 0.4166527176818345)
Тег: baking bread cookies (Similarity: 0.90, Relevance: 0.2513601040203235)
Тег: baking cookies biscuits (Similarity: 0.89, Relevance: 0.268006519464686)
Тег: baking cake oven cookies (Similarity: 0.89, Relevance: 0.3114003037472)
Тег: baking sugar cookies (Similarity: 0.89, Relevance: 0.4392340507071585)
Тег: baking cookies puff-pastry (Similarity: 0.89, Relevance: 0.4467839667414566)
Тег: baking dough sugar cookies (Similarity: 0.88, Relevance: 0.5621860698993457)
Тег: baking sugar butter cookies (Similarity: 0.87, Relevance: 0.2823559691952823)
Тег: baking cake dough sugar cookies (Similarity: 0.86, Relevance: 0.2565804791511855)


In [ ]:
import joblib
# Сохраняем модель в файл
model.save("/content/drive/My Drive/Диплом/sentence_transformer_model")


# Сохраняем эмбеддинги запросов и тегов
joblib.dump(query_embeddings, '/content/drive/My Drive/Диплом/query_embeddings.pkl')
joblib.dump(tag_embeddings, '/content/drive/My Drive/Диплом/tag_embeddings.pkl')


['/content/drive/My Drive/Диплом/tag_embeddings.pkl']

In [29]:
# Загружаем модель из файла
model = SentenceTransformer('/content/drive/My Drive/Диплом/sentence_transformer_model')

# Загружаем эмбеддинги запросов и тегов
query_embeddings = joblib.load('/content/drive/My Drive/Диплом/query_embeddings.pkl')
tag_embeddings = joblib.load('/content/drive/My Drive/Диплом/tag_embeddings.pkl')
